# **Pokemon Catch Them all**

## Objectives

* To determine which Pokemon wins the game by understanding which characteristics help them in a combat.

* The Kaggle dataset called "Pokemon- Weedle's Cave", was used as raw data and saved under the raw folder, [here]( https://www.kaggle.com/datasets/terminus7/pokemon-challenge?select=pokemon.csv)

## Inputs

* The input was "pokemon.csv" and "combats.csv"

## Outputs

* The output was "pokemon_cleaned.csv" 

## Additional Comments

* Two files were given, "pokemon.csv" has information on the Pokemon's characteristics and "combats.csv" are the results of a previous battle. This will be needed for the machine learning model to predict the winner and also to train the data.



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [8]:
import os
current_dir = os.getcwd()
print("Current working directory:", current_dir)

Current working directory: /Users/danielledelouw/Documents/code_institute/Pokemon_catch_them_all


We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [9]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [10]:
current_dir = os.getcwd()
current_dir

'/Users/danielledelouw/Documents/code_institute'

# Section 1

ETL Investigation

In [2]:
#Load python libraries
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np

Extraction: Load the Dataset

In [5]:
#read csv
print("Loading Pokemon dataset")
df = pd.read_csv('../Dataset/raw/pokemon.csv')
df

Loading Pokemon dataset


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [6]:
#read csv
print("Loading Combats dataset")
df_combats = pd.read_csv('../Dataset/raw/combats.csv')
df_combats

Loading Combats dataset


,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151
...,...,...,...
49995,707,126,707
49996,589,664,589
49997,303,368,368
49998,109,89,109
